In [1]:
import pandas as pd
import jinja2
from IPython.display import display, Javascript, HTML

> You may need to adjust the ***height*** depending on your output.

In [2]:
def inline_dc(stuff):
    """
    Embeds the HTML source of the dc charts directly into the IPython notebook.
    
    This method will not work if the dc charts depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """

    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 300px; border: none"></iframe>'.format(srcdoc=stuff.replace('"', '&quot;')))

# Import JS libraries  

> These may need to be updated as they rely on third party links

In [3]:
head = HTML("""
<head> 
 
<link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/dc/2.0.0-alpha.2/dc.css"/> 
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min.js"></script> 
<script src="https://cdnjs.cloudflare.com/ajax/libs/crossfilter/1.3.9/crossfilter.min.js"></script> 
<script src="https://cdnjs.cloudflare.com/ajax/libs/dc/1.7.0/dc.min.js"></script> 

</head>
""")

# Create Front-End

In [4]:
body_html = HTML("""


<!-- A div anchor that can be identified by id -->
<div id="piechart" class="dc-chart">
    <!-- Title or anything you want to add above the chart -->
    <strong>Pie Chart</strong>
    <!--
        This will create a reset button when ever the user selects a filter.
        Clicking on the reset button will remove all filters.
     -->
    <a class="reset" href="javascript:PieChart.filterAll();dc.redrawAll();" style="display: none;">reset</a>
    <div class="clearfix"></div>
    <!--
        dc.js will also automatically inject applied current filter value into
        any html element with css class set to "filter"
    -->
    <span class="reset" style="display: none;">Current filter: <span class="filter"></span></span>
    <br>
    </br>
</div>


""")

# Get Data

In [2]:
import scipy.io

# load .mat file into dictionary x
mat = scipy.io.loadmat('./summary_matrix.mat')

print(mat)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Sun Jun  3 20:49:06 2018', '__version__': '1.0', '__globals__': [], 'summary': array([[ (array([[1],
       [1],
       [1],
       ..., 
       [3],
       [3],
       [3]], dtype=uint8), array([[ 1],
       [ 1],
       [ 1],
       ..., 
       [41],
       [41],
       [41]], dtype=uint8), array([[array(['02-Mar-2018 13:55:47'], 
      dtype='<U20')],
       [array(['02-Mar-2018 13:55:47'], 
      dtype='<U20')],
       [array(['02-Mar-2018 13:55:47'], 
      dtype='<U20')],
       ..., 
       [array(['23-Apr-2018 13:45:54'], 
      dtype='<U20')],
       [array(['23-Apr-2018 13:45:54'], 
      dtype='<U20')],
       [array(['23-Apr-2018 13:45:54'], 
      dtype='<U20')]], dtype=object), array([[array(['02-Mar-2018 13:36:07'], 
      dtype='<U20')],
       [array(['02-Mar-2018 13:36:07'], 
      dtype='<U20')],
       [array(['02-Mar-2018 13:36:15'], 
      dtype='<U20')],
       ..., 
       [array(['23-Apr-2018 

In [4]:
print(mat['summary'].dtype)
print(mat['summary'].shape)

[('mouseid', 'O'), ('sessid', 'O'), ('sess_datetime', 'O'), ('trial_starts', 'O'), ('trial_ends', 'O'), ('led', 'O'), ('after_cue_success', 'O'), ('after_cue_drop', 'O'), ('after_cue_miss', 'O'), ('after_cue_no_pellet', 'O'), ('paw_during_wheel_after_cue_drop', 'O'), ('paw_during_wheel_after_cue_miss', 'O'), ('consumed_pellet', 'O'), ('touched_pellet', 'O'), ('cued_reach', 'O'), ('paw_during_wheel', 'O'), ('chewing_at_trial_start', 'O'), ('led_1back', 'O'), ('led_2back', 'O'), ('led_3back', 'O'), ('led_4back', 'O'), ('after_cue_success_1back', 'O'), ('after_cue_success_2back', 'O'), ('after_cue_success_3back', 'O'), ('after_cue_success_4back', 'O'), ('after_cue_drop_1back', 'O'), ('after_cue_drop_2back', 'O'), ('after_cue_drop_3back', 'O'), ('after_cue_drop_4back', 'O'), ('after_cue_miss_1back', 'O'), ('after_cue_miss_2back', 'O'), ('after_cue_miss_3back', 'O'), ('after_cue_miss_4back', 'O'), ('after_cue_no_pellet_1back', 'O'), ('after_cue_no_pellet_2back', 'O'), ('after_cue_no_pellet_

In [29]:
data = mat['summary'][0,0]
print(data['reachStarts'])
print(data['reachStarts'].shape)

[[ nan  nan  nan ...,  nan  nan  nan]
 [  0.   0.   0. ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [  0.   0.   0. ...,  nan  nan  nan]]
(3875, 1163)


1163 time points

3875 trials

In [30]:
print(data['led'].shape)
print(data['cueZone_onVoff'])

(3875, 1)
[[  0.   0.   0. ...,  nan  nan  nan]
 [  0.   0.   0. ...,  nan  nan  nan]
 [  0.   0.   0. ...,  nan  nan  nan]
 ..., 
 [  0.   0.   0. ...,  nan  nan  nan]
 [  0.   0.   0. ...,  nan  nan  nan]
 [  0.   0.   0. ...,  nan  nan  nan]]


This is when the cue started.  These should be all aligned, but take the average to find the most common start time across trials.

In [31]:
print(data['reachStarts_noPawOnWheel'])

[[ nan  nan  nan ...,  nan  nan  nan]
 [  0.   0.   0. ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [  0.   0.   0. ...,  nan  nan  nan]]


This is used as the "reaction" for computing reaction times.

In [32]:
print(data['times'])

[[   nan    nan    nan ...,    nan    nan    nan]
 [ 0.     0.035  0.07  ...,    nan    nan    nan]
 [   nan    nan    nan ...,    nan    nan    nan]
 ..., 
 [   nan    nan    nan ...,    nan    nan    nan]
 [   nan    nan    nan ...,    nan    nan    nan]
 [ 0.     0.035  0.07  ...,    nan    nan    nan]]


Since the "time" is just number of index; this matrix holds the conversion from the index to real time in seconds.

In [47]:
import numpy as np
print(np.nanargmax(np.nanmean(data['cueZone_onVoff'], axis=0)))

66


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice
  


Find the first event that is greater than 0.05 after index 66 in the reachStarts_noPawOnWheel matrix

In [89]:
x = np.nan_to_num(data['reachStarts_noPawOnWheel'][65:])
print(x.shape)
n = np.nonzero(x > 0.05)
print(n)
print(n[1].shape)
print(n[0][0])
print(n[1][0])
print(x[n[0][0]][n[1][0]])

y = np.unique(n[0], return_index=True)[1]

z = n[1]
print(z[y])

(3810, 1163)
(array([   1,    1,    2, ..., 3807, 3808, 3809]), array([ 92,  93, 155, ..., 243,  48,  73]))
(12743,)
1
92
1.0
[ 92 155 114 ...,  76  48  73]


z[y] matrix contains the indices per trial where the reaction occurred.  Now we need to convert those into actual times in seconds.

<img src="./graph_prototypes.png" width="400">
<img src="./trial_type_prototype.png" width="200">

In [5]:
df = pd.DataFrame({'population':[234,345,345,34],
                   'age':[2,3,6,9],
                   'state':['a','b','c','d'],
                   'state2':['d','c','b','a']})
df

,age,population,state,state2
0,2,234,a,d
1,3,345,b,c
2,6,345,c,b
3,9,34,d,a


# Javascript Template

In [6]:
dc = jinja2.Template(
"""   

        // Create Global Variables
        var PieChart = dc.pieChart("#piechart");

        // Load data
        var dataset = {{ data }};

        // Call function
        Graph(dataset);


// Create function
function Graph(data) {

   
    // Feed it through crossfilter  
    var ndx = crossfilter(data);
    
    // for testing
    //console.log(data);

    //define a dimension
    //Here we will group by state
    var dim = ndx.dimension(function(d) {return d.state});

    //Here we group by state and sum on column population
    var g = dim.group().reduceSum(function(d){return d.population;});
    
    //Lets create a pie chart
    PieChart.dimension(dim)
            .radius(90)
            .innerRadius(45) // used to create the donut effect
            .group(g)

            // These last two lines are not needed but are
            // here to show you how to create custom titles
            .title(function(d){ return d.data.key +": "+d.value;})
            .renderTitle(true);     

    dc.renderAll(); // render all charts on the page
    
}; // end graph function    
    
""")



# Bind data to JS Template

In [7]:
body_js = Javascript(dc.render(
    data=df.to_json(orient='records')
    ))

# Push Data to the Notebook

In [8]:
inline_dc(head.data + body_html.data + "<script>" + body_js.data + "</script>")

**Author:** [David Rojas LLC](http://hdrojas.pythonanywhere.com/)  